In [ ]:
%matplotlib inline

In [34]:
from itertools import repeat
import csv
from bs4 import BeautifulSoup
import requests

data_filepath = './newsArticlesWithLabels.tsv'
output_csv_path = 'raw_data.csv'

count = 0

NUM_SAMPLES = 10000

with open(data_filepath, newline='') as tsvin, open(output_csv_path, 'w', newline='') as csvout:
    tsvin = csv.reader(tsvin, delimiter='\t')
    csvout = csv.writer(csvout)

    for row in tsvin:
        if count < NUM_SAMPLES:
            url = row[0]
            if count > 0:
                publisher = url.split('.')[1]
                row.append(publisher)
                try:
                    req = requests.get(url)
                except requests.exceptions.RequestException as e:
                    continue
                soup = BeautifulSoup(req.text, "html.parser")
                title = soup.title.string
                if title[0] == '4' or ('Not Found' in title):
                    continue
                row.append(title)
                dem_perception = row[5]
                rep_perception = row[6]
                
                if dem_perception == "Positive":
                    dem_score = 1
                elif dem_perception == "SomewhatPositive":
                    dem_score = 0.5
                elif dem_perception == "Neutral":
                    dem_score = 0
                elif dem_perception == "SomewhatNegative":
                    dem_score = -0.5
                elif dem_perception == "Negative":
                    dem_score = -1
                else:
                    print("oops")
                
                if rep_perception == "Positive":
                    rep_score = -1
                elif rep_perception == "SomewhatPositive":
                    rep_score = -0.5
                elif rep_perception == "Neutral":
                    rep_score = 0
                elif rep_perception == "SomewhatNegative":
                    rep_score = 0.5
                elif rep_perception == "Negative":
                    rep_score = 1
                else:
                    print("oopsie")
                
                total_score = dem_score + rep_score
                
                if total_score < -1:
                    label = 1
                elif total_score < 0:
                    label = 2
                elif total_score == 0:
                    label = 3
                elif total_score <= 1:
                    label = 4
                elif total_score <= 2:
                    label = 5
                else:
                    print("woah")

                row.append(label)
            else:
                row.append("publisher")
                row.append("headline")
                row.append("avg_score")
            csvout.writerow(row)
        count += 1

KeyboardInterrupt: 

In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.7.0


In [28]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

SocialHackathon	eastus	ReelNews


In [123]:
import os
from azureml.core import Dataset, Run

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

dataset_name = 'news_data_1'

# Get a dataset by name
news_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

# Load a TabularDataset into pandas DataFrame
df = news_ds.to_pandas_dataframe()

print(df)

                                                  url  \
0   http://www.usatoday.com/story/news/nation/2013...   
1   http://www.washingtonpost.com/opinions/dont-le...   
2   http://www.foxnews.com/politics/2013/04/03/oba...   
3   http://www.breitbart.com/Big-Government/2013/1...   
4   http://www.cnn.com/2013/06/28/politics/obama-c...   
5       http://www.bbc.co.uk/news/technology-22213379   
6   http://www.nbcnews.com/technology/microsoft-le...   
7   http://www.bbc.co.uk/vietnamese/world/2013/08/...   
8   http://www.usatoday.com/story/nation/2013/08/0...   
9   http://www.dailykos.com/story/2013/12/02/12595...   
10  http://www.nytimes.com/2013/05/24/us/politics/...   
11  http://www.washingtonpost.com/local/president-...   
12  http://www.latimes.com/news/local/la-me-cap-gu...   
13  http://www.nytimes.com/2013/03/07/us/politics/...   
14  http://www.foxnews.com/politics/2013/04/15/lab...   
15  http://video.foxnews.com/v/2800623370001/are-m...   
16  http://www.cnn.com/2012/05/

In [124]:
training_data = []

for index, row in df.iterrows():
    if row['headline'] == None:
        continue
    training_data.append((row['headline'].split(), row['avg_score']))

print(training_data)

[(['Zimmerman', 'verdict', 'renews', 'focus', 'on', "'stand", 'your', "ground'", 'laws'], 3.0), (['Don’t', 'let', 'DOMA', 'fool', 'you', '—', 'the', 'Supreme', 'Court', 'is', 'restricting', 'your', 'rights', '-', 'The', 'Washington', 'Post'], 3.0), (['ObamaCare', 'in', 'Trouble?', 'Exchange', 'provision', 'delayed,', 'as', 'lawmakers', 'push', 'to', 'repeal', 'another', '|', 'Fox', 'News'], 3.0), (['Exclusive:', 'Immigration', 'Agents', 'Rip', 'House', 'Lawmakers', 'Pushing', 'Amnesty'], 3.0), (['Finalized', 'rules', 'let', 'religious', 'groups', 'opt', 'out', 'of', 'contraception', 'mandate', '-', 'CNNPolitics'], 3.0), (['Cispa', 'bill', 'on', 'cyber', 'security', 'passed', 'by', 'the', 'US', 'House', '-', 'BBC', 'News'], 3.0), (['Microsoft', 'let', 'NSA', 'bypass', 'encryption', 'on', 'mail,', 'chats', 'and', 'cloud', 'storage,', 'says', 'Guardian'], 3.0), (['Cha', 'của', 'Snowden', "'chuẩn", 'bị', 'thăm', "con'", '-', 'BBC', 'News', 'Tiếng', 'Việt'], 3.0), (['Tampa', 'socialite', 'J

In [126]:
word_to_ix = {}

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

{'Zimmerman': 0, 'verdict': 1, 'renews': 2, 'focus': 3, 'on': 4, "'stand": 5, 'your': 6, "ground'": 7, 'laws': 8, 'Don’t': 9, 'let': 10, 'DOMA': 11, 'fool': 12, 'you': 13, '—': 14, 'the': 15, 'Supreme': 16, 'Court': 17, 'is': 18, 'restricting': 19, 'rights': 20, '-': 21, 'The': 22, 'Washington': 23, 'Post': 24, 'ObamaCare': 25, 'in': 26, 'Trouble?': 27, 'Exchange': 28, 'provision': 29, 'delayed,': 30, 'as': 31, 'lawmakers': 32, 'push': 33, 'to': 34, 'repeal': 35, 'another': 36, '|': 37, 'Fox': 38, 'News': 39, 'Exclusive:': 40, 'Immigration': 41, 'Agents': 42, 'Rip': 43, 'House': 44, 'Lawmakers': 45, 'Pushing': 46, 'Amnesty': 47, 'Finalized': 48, 'rules': 49, 'religious': 50, 'groups': 51, 'opt': 52, 'out': 53, 'of': 54, 'contraception': 55, 'mandate': 56, 'CNNPolitics': 57, 'Cispa': 58, 'bill': 59, 'cyber': 60, 'security': 61, 'passed': 62, 'by': 63, 'US': 64, 'BBC': 65, 'Microsoft': 66, 'NSA': 67, 'bypass': 68, 'encryption': 69, 'mail,': 70, 'chats': 71, 'and': 72, 'cloud': 73, 'stora

In [30]:
experiment_name = 'reel-news-3'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [31]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [32]:
import os
script_folder = os.path.join(os.getcwd(), "reel-news")
os.makedirs(script_folder, exist_ok=True)

In [137]:
print(len(word_to_ix))

339


In [26]:
%%writefile $script_folder/train.py

import argparse
import os
import numpy as np
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.linear_model import LogisticRegression
import joblib

from azureml.core import Run

# let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)

dataset_name = 'news_data_1'

# Get a dataset by name
news_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

# Load a TabularDataset into pandas DataFrame
df = news_ds.to_pandas_dataframe()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else -1 for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [
    ("ObamaCare in Trouble? Exchange provision delayed, as lawmakers push to repeal another | Fox News".split(), 3),
    ("Obama is bad".split(), 5),
    ("I love Obama".split(), 1),
    ("I love Trump".split(), 5),
    ("guns are bad".split(), 1),
    ("I love guns".split(), 5)
]

word_to_ix = {}


for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.fc1 = nn.Linear(hidden_dim, 100)
        self.fc2 = nn.Linear(100, output_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        _, (hidden_text, _) = self.lstm(embeds.view(len(sentence), 1, -1))
        hidden_text = nn.ReLU()(hidden_text)
        prediction = self.fc1(hidden_text.squeeze(0))
        prediction = nn.ReLU()(prediction)
        prediction = self.fc2(prediction)
        return prediction

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# get hold of the current run
run = Run.get_context()

model = LSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), 5)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(1000):  # again, normally you would NOT do 300 epochs, it is toy data
    preds = torch.tensor([]).long()
    correct = 0
    total = 0
    total_loss = 0
    for sentence, labels in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        labels = torch.tensor(labels - 1).unsqueeze(0)

        # Step 3. Run our forward pass.
        outputs = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = torch.argmax(outputs, axis=1)
        if epoch == 500:
            print(sentence)
            print(outputs)
            print(pred)
            print(labels)
        preds = torch.cat((preds, pred))
        # print(outputs)
        # print(pred)
        # print(labels)
        correct += (torch.sum(pred == labels)).item()
        total += len(labels)
        total_loss += loss.item()


# print('Train a logistic regression model with regularization rate of', args.reg)
# clf = LogisticRegression(C=1.0/args.reg, solver="liblinear", multi_class="auto", random_state=42)
# clf.fit(X_train, y_train)

# print('Predict the test set')
# y_hat = clf.predict(X_test)

# # calculate accuracy on the prediction
# acc = np.average(y_hat == y_test)
# print('Accuracy is', acc)

# run.log('regularization rate', np.float(args.reg))
# run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)

FINAL_MODEL_PATH = 'outputs/final_LSTM_model.pkl'

joblib.dump(value=model, filename=FINAL_MODEL_PATH)

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/testing123/code/Users/pillaip2/reel-news/train.py


In [58]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataprep[pandas,fuse]>=1.1.14', 'azureml-defaults', 'torch'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
      

In [14]:
from azureml.train.estimator import Estimator

script_params = {
    # to mount files referenced by mnist dataset
    '--data-folder': './raw_data.csv'
}

est = Estimator(source_directory=script_folder,
              script_params=script_params,
              compute_target=compute_target,
              environment_definition=env,
              entry_script='train.py')

In [15]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
reel-news-1,reel-news-1_1593323545_c2df1efd,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
run.wait_for_completion(show_output=True) 

RunId: reel-news-1_1593323545_c2df1efd
Web View: https://ml.azure.com/experiments/reel-news-1/runs/reel-news-1_1593323545_c2df1efd?wsid=/subscriptions/b5f11446-108e-46f8-838e-22ef8264534d/resourcegroups/ReelNews/workspaces/SocialHackathon

Streaming azureml-logs/55_azureml-execution-tvmps_d3d2130281d10564d3e6d8649d8653f8fd8080a14186e7408da0900ad09d540c_d.txt

2020-06-28T05:56:33Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-06-28T05:56:33Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
2020-06-28T05:56:33Z Starting output-watcher...
2020-06-28T05:56:33Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_aeaad50a14947a3cab4630c670d5ecfa
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling 

{'runId': 'reel-news-1_1593323545_c2df1efd',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-06-28T05:56:31.183756Z',
 'endTimeUtc': '2020-06-28T06:01:46.639556Z',
 'warnings': [{'message': "This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.\nPlease enable Docker in the environment section of your run configuration to stop seeing this warning message."}],
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '980821ec-2f0c-4e5e-bea1-55a51da075e7',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': ['--data-folder', './raw_data.csv'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName

In [17]:
MODEL_NAME = 'LSTM_bias_model'
FINAL_MODEL_PATH = 'outputs/final_LSTM_model.pkl'

model = run.register_model(model_name=MODEL_NAME, model_path=FINAL_MODEL_PATH)

In [133]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# training_data = [
#     ("Trump is bad".split(), 2),
#     ("Obama is bad".split(), 4),
#     ("I love Obama".split(), 1),
#     ("I love Trump".split(), 5),
#     ("guns are bad".split(), 1),
#     ("I love guns".split(), 5)
# ]

# word_to_ix = {}


# for sent, tags in training_data:
#     for word in sent:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# print(word_to_ix)

EMBEDDING_DIM = 16
HIDDEN_DIM = 16

In [129]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.fc1 = nn.Linear(hidden_dim, 100)
        self.fc2 = nn.Linear(100, output_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        _, (hidden_text, _) = self.lstm(embeds.view(len(sentence), 1, -1))
        hidden_text = nn.ReLU()(hidden_text)
        prediction = self.fc1(hidden_text.squeeze(0))
        prediction = nn.ReLU()(prediction)
        prediction = self.fc2(prediction)
        return prediction

In [135]:
model = LSTM(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), 5)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(200):  # again, normally you would NOT do 300 epochs, it is toy data
    preds = torch.tensor([]).long()
    correct = 0
    total = 0
    total_loss = 0
    for sentence, labels in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        labels = torch.tensor(labels - 1).unsqueeze(0)
        labels = labels.long()

        # Step 3. Run our forward pass.
        outputs = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = torch.argmax(outputs, axis=1)
        if epoch == 500:
            print(sentence)
            print(outputs)
            print(pred)
            print(labels)
        preds = torch.cat((preds, pred))
        # print(outputs)
        # print(pred)
        # print(labels)
        correct += (torch.sum(pred == labels)).item()
        total += len(labels)
        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, avg_loss: {total_loss / total}, accuracy: {correct / total}')

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[ 0.0609, -0.0953,  0.1022,  0.0538, -0.0884]])
Epoch: 0, avg_loss: 1.5769740152359009, accuracy: 0.48
Epoch: 10, avg_loss: 1.3939677214622497, accuracy: 0.48
Epoch: 20, avg_loss: 1.2298057556152344, accuracy: 0.48
Epoch: 30, avg_loss: 1.1455524396896362, accuracy: 0.48
Epoch: 40, avg_loss: 1.0937681877613068, accuracy: 0.48
Epoch: 50, avg_loss: 1.0412694388628005, accuracy: 0.52
Epoch: 60, avg_loss: 0.9834513092041015, accuracy: 0.56
Epoch: 70, avg_loss: 0.9179824233055115, accuracy: 0.64
Epoch: 80, avg_loss: 0.8424261593818665, accuracy: 0.64
Epoch: 90, avg_loss: 0.7612947103381157, accuracy: 0.7
Epoch: 100, avg_loss: 0.6763800674676895, accuracy: 0.7
Epoch: 110, avg_loss: 0.5936621616780758, accuracy: 0.74
Epoch: 120, avg_loss: 0.5136321193724871, accuracy: 0.84
Epoch: 130, avg_loss: 0.444378707036376, accuracy: 0.84
Epoch: 140, avg_loss: 0.38354562401771547, accuracy: 0.84
Epoch: 150, avg_loss: 0.33215956687927245, accuracy: 0.88
Epoch: 160, avg_loss: 0.29063264179974796, a

In [53]:
def get_prediction(sentence):
    return torch.argmax(model(prepare_sequence(sentence.split(), word_to_ix))) + 1

tensor(5)


In [136]:
torch.save(model.state_dict(), './FINAL_LSTM')

In [74]:
import joblib

FINAL_MODEL_PATH = 'final_LSTM_model.pkl'

joblib.dump(value=model, filename=FINAL_MODEL_PATH)

['final_LSTM_model.pkl']

In [109]:
from azureml.core import Workspace
ws = Workspace.from_config()

from azureml.core.model import Model

MODEL_NAME = 'LSTM_bias_model'

model_name = MODEL_NAME
model = Model.register(model_path="final_LSTM_model.pkl",
                        model_name=model_name,
                        tags={"model": "classification"},
                        description="LSTM Bias Detection",
                        workspace=ws)

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-defaults', 'pandas', 'torch', 'joblib', 'numpy'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

Registering model LSTM_bias_model


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
      

In [108]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.7.0


In [120]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from azureml.core import Workspace, Dataset

class LSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.fc1 = nn.Linear(hidden_dim, 100)
        self.fc2 = nn.Linear(100, output_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        _, (hidden_text, _) = self.lstm(embeds.view(len(sentence), 1, -1))
        hidden_text = nn.ReLU()(hidden_text)
        prediction = self.fc1(hidden_text.squeeze(0))
        prediction = nn.ReLU()(prediction)
        prediction = self.fc2(prediction)
        return prediction

def init():
    global model
    global word_to_ix
    # subscription_id = 'b5f11446-108e-46f8-838e-22ef8264534d'
    # resource_group = 'ReelNews'
    # workspace_name = 'SocialHackathon'
    # # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # # For multiple models, it points to the folder containing all deployed models (./azureml-models)

    # ws = Workspace(subscription_id, resource_group, workspace_name)

    # dataset_name = 'news_data_1'
    # # Get a dataset by name
    # news_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

    # data_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'raw_data.csv')
    # df = pd.read_csv(data_path)

    # # Load a TabularDataset into pandas DataFrame
    # # df = news_ds.to_pandas_dataframe()

    # training_data = []

    # for index, row in df.iterrows():
    #     if row['headline'] == None:
    #         continue
    #     training_data.append((row['headline'].split(), row['avg_score']))

    training_data = [
        ("Trump is bad".split(), 1),
        ("Obama is bad".split(), 5),
        ("I love Obama".split(), 1),
        ("I love Trump".split(), 5),
        ("guns are bad".split(), 1),
        ("I love guns".split(), 5)
    ]

    word_to_ix = {}

    for sent, tags in training_data:
        for word in sent:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)

    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'final_LSTM_model.pkl')
    model = joblib.load(model_path)

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else -1 for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def run(raw_data):
    data = json.loads(raw_data)['data']
    # make prediction
    y_hat = torch.argmax(model(prepare_sequence(data.split(), word_to_ix))) + 1
    # you can return any data type as long as it is JSON-serializable
    return y_hat

Overwriting score.py


In [111]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"method" : "LSTM"}, 
                                               description='Predict bias with Pytorch LSTM')

In [112]:
print(run.get_file_names())
print(run)

['azureml-logs/55_azureml-execution-tvmps_d3d2130281d10564d3e6d8649d8653f8fd8080a14186e7408da0900ad09d540c_d.txt', 'azureml-logs/65_job_prep-tvmps_d3d2130281d10564d3e6d8649d8653f8fd8080a14186e7408da0900ad09d540c_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_d3d2130281d10564d3e6d8649d8653f8fd8080a14186e7408da0900ad09d540c_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/102_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/final_LSTM_model.pkl']
Run(Experiment: reel-news-1,
Id: reel-news-1_1593323545_c2df1efd,
Type: azureml.scriptrun,
Status: Completed)


In [121]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, MODEL_NAME)

inference_config = InferenceConfig(entry_script="score.py", environment=env)

service = Model.deploy(workspace=ws, 
                       name='reel-news-svc-v10', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Running...........................................................................

KeyboardInterrupt: 

In [122]:
service.get_logs()

'2020-06-28T15:57:25,098357513+00:00 - rsyslog/run \n2020-06-28T15:57:25,098636323+00:00 - gunicorn/run \n2020-06-28T15:57:25,099647358+00:00 - iot-server/run \n2020-06-28T15:57:25,109087485+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_b52b1e4b9e6fc311fbce240c1d69a6ed/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b52b1e4b9e6fc311fbce240c1d69a6ed/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b52b1e4b9e6fc311fbce240c1d69a6ed/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b52b1e4b9e6fc311fbce240c1d69a6ed/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b52b1e4b9e6fc311fbce240c1d69a6ed/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

# And that's it!